In [2]:
import numpy as np
import pandas as pd

In [3]:
import torch
import torch.nn.init as init

In [4]:
# hemibrain
!ls exported-traced-adjacencies 

ls: cannot access 'exported-traced-adjacencies': No such file or directory


In [5]:
traced_neurons = pd.read_csv('exported-traced-adjacencies/traced-neurons.csv')
traced_roi_connections = pd.read_csv('exported-traced-adjacencies/traced-roi-connections.csv')
traced_total_connections = pd.read_csv('exported-traced-adjacencies/traced-total-connections.csv')

FileNotFoundError: ignored

In [ ]:
traced_neurons

,bodyId,instance,type
0,5813054846,PVM03y_pct(PVM03)_R,PVM03y_pct
1,1479492693,AVL13t_pct(AVL13)_R,AVL13t_pct
2,5812982999,KCg,KCg
3,1720300275,AVL04v_d_pct(AVL04)_R,AVL04v_d_pct
4,5813060766,PDL14m_a_pct(PDL14)_R,PDL14m_a_pct
...,...,...,...
21658,583132200,(ADM01)_L,NaN
21659,5813098080,ADL01os_pct(ADL01)_R,ADL01os_pct
21660,1948478919,AVL14t_pct(AVL14)_R,AVL14t_pct
21661,604061479,AVL12j_pct(AVL12)_R,AVL12j_pct


In [ ]:
num_nodes = 21663

In [ ]:
unique_labels = traced_neurons['instance'].unique()

In [ ]:
# assign indexes
label_idx_dict = {}
for i, label in enumerate(unique_labels):
    label_idx_dict[label] = i

In [ ]:
labels = [label_idx_dict[label] for label in traced_neurons['instance'].values]

In [ ]:
# get number of neuron types
num_classes = traced_neurons['instance'].nunique()
print(num_classes)

6223


In [ ]:
traced_total_connections

,bodyId_pre,bodyId_post,weight
0,5813054846,5813020143,1
1,5813054846,2215550458,1
2,5813054846,1448575109,1
3,5813054846,1223388206,1
4,5813054846,1913403687,1
...,...,...,...
3413155,453794660,481070027,1
3413156,453794660,423792440,2
3413157,453794660,455172836,1
3413158,453794660,388975834,3


In [ ]:
# assign indexes to bodyIds
bodyIds = traced_total_connections['bodyId_post'].unique()
bodyIds = sorted(bodyIds)

In [ ]:
def bodyId_to_idx(bodyId):
    return bodyIds.index(bodyId)

# Convert to DGL graph object

In [ ]:
import os
os.environ['DGLBACKEND'] = 'pytorch'
import torch
import torch.nn as nn
import torch.nn.functional as F

import dgl
import dgl.data
from dgl.utils import expand_as_pair
import dgl.function as fn

import timeit

In [ ]:
bodyId_idx_dict = {}
for i, bodyId in enumerate(bodyIds):
    bodyId_idx_dict[bodyId] = i

In [ ]:
start = timeit.default_timer()
pre_indexes = np.vectorize(bodyId_idx_dict.get)(traced_total_connections['bodyId_pre'].values)
end = timeit.default_timer()
print(f"time elapsed: {end - start} s")
# from https://stackoverflow.com/questions/16992713/translate-every-element-in-numpy-array-according-to-key

time elapsed: 3.984710899999982 s


In [ ]:
start = timeit.default_timer()
post_indexes = np.vectorize(bodyId_idx_dict.get)(traced_total_connections['bodyId_post'].values)
end = timeit.default_timer()
print(f"time elapsed: {end - start} s")
# from https://stackoverflow.com/questions/16992713/translate-every-element-in-numpy-array-according-to-key

time elapsed: 5.17346520000001 s


In [ ]:
start = timeit.default_timer()
graph = dgl.graph((pre_indexes, post_indexes), num_nodes=num_nodes)
end = timeit.default_timer()
print(f"time elapsed: {end-start}s")

time elapsed: 0.10473259999992024s


In [ ]:
graph

Graph(num_nodes=21663, num_edges=3413160,
      ndata_schemes={}
      edata_schemes={})

### using GCN
model from https://discuss.dgl.ai/t/how-could-use-edge-or-node-weight/1310/2

In [ ]:
e_weights = torch.tensor(traced_total_connections['weight'].values)#.unsqueeze(-1)

In [ ]:
e_weights.shape

torch.Size([3413160])

In [ ]:
# pylint: disable=W0235
class GraphConv(nn.Module):
    r"""
    Description
    -----------
    Graph convolution was introduced in `GCN <https://arxiv.org/abs/1609.02907>`__
    and mathematically is defined as follows:
    .. math::
      h_i^{(l+1)} = \sigma(b^{(l)} + \sum_{j\in\mathcal{N}(i)}\frac{1}{c_{ij}}h_j^{(l)}W^{(l)})
    where :math:`\mathcal{N}(i)` is the set of neighbors of node :math:`i`,
    :math:`c_{ij}` is the product of the square root of node degrees
    (i.e.,  :math:`c_{ij} = \sqrt{|\mathcal{N}(i)|}\sqrt{|\mathcal{N}(j)|}`),
    and :math:`\sigma` is an activation function.

    Parameters
    ----------
    in_feats : int
        Input feature size; i.e, the number of dimensions of :math:`h_j^{(l)}`.
    out_feats : int
        Output feature size; i.e., the number of dimensions of :math:`h_i^{(l+1)}`.
    norm : str, optional
        How to apply the normalizer. If is `'right'`, divide the aggregated messages
        by each node's in-degrees, which is equivalent to averaging the received messages.
        If is `'none'`, no normalization is applied. Default is `'both'`,
        where the :math:`c_{ij}` in the paper is applied.
    weight : bool, optional
        If True, apply a linear layer. Otherwise, aggregating the messages
        without a weight matrix.
    bias : bool, optional
        If True, adds a learnable bias to the output. Default: ``True``.
    activation : callable activation function/layer or None, optional
        If not None, applies an activation function to the updated node features.
        Default: ``None``.
    allow_zero_in_degree : bool, optional
        If there are 0-in-degree nodes in the graph, output for those nodes will be invalid
        since no message will be passed to those nodes. This is harmful for some applications
        causing silent performance regression. This module will raise a DGLError if it detects
        0-in-degree nodes in input graph. By setting ``True``, it will suppress the check
        and let the users handle it by themselves. Default: ``False``.

    Attributes
    ----------
    weight : torch.Tensor
        The learnable weight tensor.
    bias : torch.Tensor
        The learnable bias tensor.

    Note
    ----
    Zero in-degree nodes will lead to invalid output value. This is because no message
    will be passed to those nodes, the aggregation function will be appied on empty input.
    A common practice to avoid this is to add a self-loop for each node in the graph if
    it is homogeneous, which can be achieved by:
    >>> g = ... # a DGLGraph
    >>> g = dgl.add_self_loop(g)
    Calling ``add_self_loop`` will not work for some graphs, for example, heterogeneous graph
    since the edge type can not be decided for self_loop edges. Set ``allow_zero_in_degree``
    to ``True`` for those cases to unblock the code and handle zere-in-degree nodes manually.
    A common practise to handle this is to filter out the nodes with zere-in-degree when use
    after conv.

    Examples
    --------
    >>> import dgl
    >>> import numpy as np
    >>> import torch as th
    >>> from dgl.nn import GraphConv
    >>> # Case 1: Homogeneous graph
    >>> g = dgl.graph(([0,1,2,3,2,5], [1,2,3,4,0,3]))
    >>> g = dgl.add_self_loop(g)
    >>> feat = th.ones(6, 10)
    >>> conv = GraphConv(10, 2, norm='both', weight=True, bias=True)
    >>> res = conv(g, feat)
    >>> print(res)
    tensor([[ 1.3326, -0.2797],
            [ 1.4673, -0.3080],
            [ 1.3326, -0.2797],
            [ 1.6871, -0.3541],
            [ 1.7711, -0.3717],
            [ 1.0375, -0.2178]], grad_fn=<AddBackward0>)
    >>> # allow_zero_in_degree example
    >>> g = dgl.graph(([0,1,2,3,2,5], [1,2,3,4,0,3]))
    >>> conv = GraphConv(10, 2, norm='both', weight=True, bias=True, allow_zero_in_degree=True)
    >>> res = conv(g, feat)
    >>> print(res)
    tensor([[-0.2473, -0.4631],
            [-0.3497, -0.6549],
            [-0.3497, -0.6549],
            [-0.4221, -0.7905],
            [-0.3497, -0.6549],
            [ 0.0000,  0.0000]], grad_fn=<AddBackward0>)
    >>> # Case 2: Unidirectional bipartite graph
    >>> u = [0, 1, 0, 0, 1]
    >>> v = [0, 1, 2, 3, 2]
    >>> g = dgl.bipartite((u, v))
    >>> u_fea = th.rand(2, 5)
    >>> v_fea = th.rand(4, 5)
    >>> conv = GraphConv(5, 2, norm='both', weight=True, bias=True)
    >>> res = conv(g, (u_fea, v_fea))
    >>> res
    tensor([[-0.2994,  0.6106],
            [-0.4482,  0.5540],
            [-0.5287,  0.8235],
            [-0.2994,  0.6106]], grad_fn=<AddBackward0>)
    """
    def __init__(self,
                 in_feats,
                 out_feats,
                 norm='both',
                 weight=True,
                 bias=True,
                 activation=None,
                 allow_zero_in_degree=False):
        super(GraphConv, self).__init__()
        if norm not in ('none', 'both', 'right'):
            raise DGLError('Invalid norm value. Must be either "none", "both" or "right".'
                           ' But got "{}".'.format(norm))
        self._in_feats = in_feats
        self._out_feats = out_feats
        self._norm = norm
        self._allow_zero_in_degree = allow_zero_in_degree

        if weight:
            self.weight = nn.Parameter(torch.Tensor(in_feats, out_feats))
        else:
            self.register_parameter('weight', None)

        if bias:
            self.bias = nn.Parameter(torch.Tensor(out_feats))
        else:
            self.register_parameter('bias', None)

        self.reset_parameters()

        self._activation = activation

    def reset_parameters(self):
        r"""
        Description
        -----------
        Reinitialize learnable parameters.

        Note
        ----
        The model parameters are initialized as in the
        `original implementation <https://github.com/tkipf/gcn/blob/master/gcn/layers.py>`__
        where the weight :math:`W^{(l)}` is initialized using Glorot uniform initialization
        and the bias is initialized to be zero.
        """
        if self.weight is not None:
            init.xavier_uniform_(self.weight)
        if self.bias is not None:
            init.zeros_(self.bias)

    def set_allow_zero_in_degree(self, set_value):
        r"""
        Description
        -----------
        Set allow_zero_in_degree flag.

        Parameters
        ----------
        set_value : bool
            The value to be set to the flag.
        """
        self._allow_zero_in_degree = set_value

    def forward(self, graph, feat, eweight, weight=None):
        r"""
        Description
        -----------
        Compute graph convolution.

        Parameters
        ----------
        graph : DGLGraph
            The graph.
        feat : torch.Tensor or pair of torch.Tensor
            If a torch.Tensor is given, it represents the input feature of shape
            :math:`(N, D_{in})`
            where :math:`D_{in}` is size of input feature, :math:`N` is the number of nodes.
            If a pair of torch.Tensor is given, which is the case for bipartite graph, the pair
            must contain two tensors of shape :math:`(N_{in}, D_{in_{src}})` and
            :math:`(N_{out}, D_{in_{dst}})`.
        eweight : torch.Tensor of shape (E, 1)
            Edge weights, E for the number of edges.
        weight : torch.Tensor, optional
            Optional external weight tensor.

        Returns
        -------
        torch.Tensor
            The output feature

        Raises
        ------
        DGLError
            Case 1:
            If there are 0-in-degree nodes in the input graph, it will raise DGLError
            since no message will be passed to those nodes. This will cause invalid output.
            The error can be ignored by setting ``allow_zero_in_degree`` parameter to ``True``.
            Case 2:
            External weight is provided while at the same time the module
            has defined its own weight parameter.

        Note
        ----
        * Input shape: :math:`(N, *, \text{in_feats})` where * means any number of additional
          dimensions, :math:`N` is the number of nodes.
        * Output shape: :math:`(N, *, \text{out_feats})` where all but the last dimension are
          the same shape as the input.
        * Weight shape: :math:`(\text{in_feats}, \text{out_feats})`.
        """
        with graph.local_scope():
            if not self._allow_zero_in_degree:
                if (graph.in_degrees() == 0).any():
                    raise DGLError('There are 0-in-degree nodes in the graph, '
                                   'output for those nodes will be invalid. '
                                   'This is harmful for some applications, '
                                   'causing silent performance regression. '
                                   'Adding self-loop on the input graph by '
                                   'calling `g = dgl.add_self_loop(g)` will resolve '
                                   'the issue. Setting ``allow_zero_in_degree`` '
                                   'to be `True` when constructing this module will '
                                   'suppress the check and let the code run.')

            # (BarclayII) For RGCN on heterogeneous graphs we need to support GCN on bipartite.
            feat_src, feat_dst = expand_as_pair(feat, graph)
            if self._norm == 'both':
                degs = graph.out_degrees().float().clamp(min=1)
                norm = torch.pow(degs, -0.5)
                shp = norm.shape + (1,) * (feat_src.dim() - 1)
                norm = torch.reshape(norm, shp)
                feat_src = feat_src * norm

            if weight is not None:
                if self.weight is not None:
                    raise DGLError('External weight is provided while at the same time the'
                                   ' module has defined its own weight parameter. Please'
                                   ' create the module with flag weight=False.')
            else:
                weight = self.weight

            # Set edge weights
            graph.edata['w'] = eweight
            if self._in_feats > self._out_feats:
                # mult W first to reduce the feature size for aggregation.
                if weight is not None:
                    feat_src = torch.matmul(feat_src, weight)
                graph.srcdata['h'] = feat_src
                # Changed from fn.copy_src to fn.u_mul_e
                graph.update_all(fn.u_mul_e(lhs_field='h', rhs_field='w', out='m'),
                                 fn.sum(msg='m', out='h'))
                rst = graph.dstdata['h']
            else:
                # aggregate first then mult W
                graph.srcdata['h'] = feat_src
                # Changed from fn.copy_src to fn.u_mul_e
                graph.update_all(fn.u_mul_e(lhs_field='h', rhs_field='w', out='m'),
                                 fn.sum(msg='m', out='h'))
                rst = graph.dstdata['h']
                if weight is not None:
                    rst = torch.matmul(rst, weight)

            if self._norm != 'none':
                degs = graph.in_degrees().float().clamp(min=1)
                if self._norm == 'both':
                    norm = torch.pow(degs, -0.5)
                else:
                    norm = 1.0 / degs
                shp = norm.shape + (1,) * (feat_dst.dim() - 1)
                norm = torch.reshape(norm, shp)
                rst = rst * norm

            if self.bias is not None:
                rst = rst + self.bias

            if self._activation is not None:
                rst = self._activation(rst)

            return rst

In [ ]:
gnn = GraphConv(in_feats = 2, # in degree and out degree
               out_feats = num_classes,
               allow_zero_in_degree = True)

In [ ]:
# add additional parameters

In [ ]:
graph.ndata['feat'] = torch.stack([graph.in_degrees(), graph.out_degrees()]).T

In [ ]:
graph.ndata['train_mask'] = torch.zeros(num_nodes, dtype=bool)

In [ ]:
graph.ndata['test_mask'] = torch.zeros(num_nodes, dtype=bool)
graph.ndata['val_mask'] = torch.zeros(num_nodes, dtype=bool)


In [ ]:
graph.ndata['train_mask'][0:15000] = True
graph.ndata['val_mask'][15000:16000] = True
graph.ndata['test_mask'][16000:] = True

In [ ]:
graph.ndata['label'] = torch.tensor(labels)

In [ ]:
def train(g, model, lr = 0.001):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata["feat"]
    labels = g.ndata["label"]
    train_mask = g.ndata["train_mask"]
    val_mask = g.ndata["val_mask"]
    test_mask = g.ndata["test_mask"]
    for e in range(100):
        # Forward
        logits = model(g, features, e_weights.to(torch.float32))

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        
#         print(logits[train_mask].squeeze().shape)
#         print(logits[train_mask].squeeze())
#         print(labels[train_mask].shape)
#         print(labels[train_mask])
        
#         loss = F.cross_entropy(logits[train_mask], labels[train_mask])
        loss = F.cross_entropy(logits[train_mask].squeeze(0), labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print(
                "In epoch {}, loss: {:.3f}, train acc: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})".format(
                    e, loss, train_acc, val_acc, best_val_acc, test_acc, best_test_acc
                )
            )


train(graph, gnn)

C:\Users\Ben Wu\.conda\envs\gnn\lib\site-packages\dgl\backend\pytorch\tensor.py:445: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (


In epoch 0, loss: 47.461, train acc: 0.000, val acc: 0.000 (best 0.000), test acc: 0.000 (best 0.000)
In epoch 5, loss: 34.969, train acc: 0.002, val acc: 0.003 (best 0.003), test acc: 0.003 (best 0.003)
In epoch 10, loss: 26.599, train acc: 0.012, val acc: 0.012 (best 0.012), test acc: 0.011 (best 0.014)
In epoch 15, loss: 20.647, train acc: 0.004, val acc: 0.001 (best 0.012), test acc: 0.004 (best 0.014)
In epoch 20, loss: 16.095, train acc: 0.002, val acc: 0.003 (best 0.012), test acc: 0.001 (best 0.014)
